In [2]:
from functools import partial
from datasets import load_dataset
from transformers import AutoTokenizer
from difflib import SequenceMatcher

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/xuyixuan/opt/anaconda3/envs/goldfish/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
unmatch = load_dataset(
    "json",
    data_files="/Users/xuyixuan/Downloads/Project/PDM/PDM/data/gutenberg_en_8k/unmatch.jsonl",  
    split="train",
)

Generating train split: 1231 examples [00:00, 4617.35 examples/s]


In [13]:
unmatch

Dataset({
    features: ['text', 'input_ids'],
    num_rows: 1231
})

In [14]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.model_max_length = 200_000

In [15]:
tokenize_fn = partial(
   tokenizer, 
   truncation=False, 
   padding=False, 
   add_special_tokens=False
)

In [16]:
def find_unmatch(unmatch, idx):
    a = tokenize_fn(text=unmatch[idx]['text']).input_ids
    b = unmatch[idx]['input_ids']
    s = SequenceMatcher(None, a, b)
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'replace':
            print('{:<7} a[{:>4}:{:>4}] --> b[{:>4}:{:>4}] {!r:>12} --> {!r:<20} Replace [{:^20}] with [{:^20}]'.format(
                tag, i1, i2, j1, j2, a[i1:i2], b[j1:j2], tokenizer.decode(a[i1:i2]), tokenizer.decode(b[j1:j2])))


In [34]:
find_unmatch(unmatch, 112)

replace a[3199:3201] --> b[3199:3201]  [596, 1002] --> [364, 68182]         Replace [       'spect       ] with [       'spect       ]
replace a[5007:5009] --> b[5007:5010]  [596, 2972] --> [364, 2203, 974]     Replace [       'spose       ] with [       'spose       ]


In [31]:
tokenizer.decode([364, 588])

"'ve"